In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity, rbf_kernel
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report

/home/rinat/repos/galki_rutube/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
TRAIN_DATA_PATH = "src/data/train.parquet"
ALL_COLUMNS = ["Вопрос пользователя", "Классификатор 1 уровня", "Классификатор 2 уровня", "target"]

CLASSIFIER_MODEL_NAME = "intfloat/multilingual-e5-large-instruct"

TARGET_COLUMN = "target"

QUESTION_COLUMN = "Вопрос пользователя"
EMBEDDING_COLUMN = "embedding"

TOP_N = 1

In [8]:
train = pd.read_parquet('/home/rinat/repos/galki_rutube/src/data/train.parquet')

In [9]:
model = SentenceTransformer(CLASSIFIER_MODEL_NAME)

In [10]:

def find_similar_class(query: str, train: pd.DataFrame, model: SentenceTransformer, top_n: int):
    query_embedding = model.encode([query])
    
    try:
        similarities = cosine_similarity(query_embedding, np.array(train[EMBEDDING_COLUMN].tolist()))
    except ValueError as e:
        return [], f"Ошибка в вычислении сходства: {e}"

    similar_indices = np.argsort(similarities[0])[::-1][:top_n]
    similar_classes = train.iloc[similar_indices][TARGET_COLUMN]
    similar_questions = train.iloc[similar_indices][QUESTION_COLUMN]
    most_common_class = Counter(similar_classes).most_common(1)[0][0]
    
    return similar_questions, most_common_class

In [11]:
similar_questions, predicted_class = find_similar_class("Что такое метка 18+?", train, model, TOP_N)

In [14]:
similar_example = similar_questions.iloc[0] if not similar_questions.empty else None

In [24]:
import asyncio
import numpy as np
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import uvicorn

class Request(BaseModel):
    question: str

class Response(BaseModel):
    class_: str


In [25]:
response = Response(
    class_=predicted_class
)

In [26]:
response

Response(class_='МОДЕРАЦИЯ_Смена категории/возрастные ограничения')

In [22]:
similar_questions

6338    Что за метка 18+?
Name: Вопрос пользователя, dtype: object

In [19]:
similar_example

'Что за метка 18+?'

In [15]:
similar_example

'Что за метка 18+?'

In [12]:
similar_questions

6338    Что за метка 18+?
Name: Вопрос пользователя, dtype: object

In [13]:
predicted_class

'МОДЕРАЦИЯ_Смена категории/возрастные ограничения'

In [ ]:
def predict_class(request: Request):
    query = request.question
    similar_questions, predicted_class = find_similar_class(query, train, model, TOP_N)
    similar_example = similar_questions.iloc[0] if not similar_questions.empty else None
    
    response = Response(
        class_=predicted_class,
        similar_example=similar_example[QUESTION_COLUMN] if similar_example is not None else None
    )
    return response

def find_similar_class(query: str, train: pd.DataFrame, model: SentenceTransformer, top_n: int):
    query_embedding = model.encode([query])
    
    try:
        similarities = cosine_similarity(query_embedding, np.array(train[EMBEDDING_COLUMN].tolist()))
    except ValueError as e:
        return [], f"Ошибка в вычислении сходства: {e}"

    similar_indices = np.argsort(similarities[0])[::-1][:top_n]
    similar_classes = train.iloc[similar_indices][TARGET_COLUMN]
    similar_questions = train.iloc[similar_indices][QUESTION_COLUMN]
    most_common_class = Counter(similar_classes).most_common(1)[0][0]
    
    return similar_questions, most_common_class

In [5]:
train

,Вопрос пользователя,Классификатор 1 уровня,Классификатор 2 уровня,target,embedding
0,Здравствуйте! Можно уточнить причины Правилhtt...,МОДЕРАЦИЯ,Отклонение/блокировка видео,МОДЕРАЦИЯ_Отклонение/блокировка видео,"[0.0062491735, -0.010457837, 0.008955387, -0.0..."
1,"Добрый вечер, какой топ причин блокировки виде...",МОДЕРАЦИЯ,Отклонение/блокировка видео,МОДЕРАЦИЯ_Отклонение/блокировка видео,"[0.02596741, 0.0012711333, -0.014978344, -0.03..."
2,"Все пишут, что монетизация на рутубе отключает...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,МОНЕТИЗАЦИЯ_Отключение/подключение монетизации,"[-0.0012037684, 0.013548447, -0.013419978, -0...."
3,Что запрещено в монетизации и что можно выклад...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,МОНЕТИЗАЦИЯ_Отключение/подключение монетизации,"[0.012090996, 0.0013656222, -0.010803133, -0.0..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,МОНЕТИЗАЦИЯ_Отключение/подключение монетизации,"[-0.009512662, -0.009229696, -0.018809447, -0...."
...,...,...,...,...,...
8703,Как на Rutube можно найти и смотреть короткие ...,ВИДЕО,Воспроизведение видео,ВИДЕО_Воспроизведение видео,"[-0.0016850237, 0.010049397, -0.01154559, -0.0..."
8704,"Есть ли на Rutube раздел с короткими роликами,...",ВИДЕО,Воспроизведение видео,ВИДЕО_Воспроизведение видео,"[-0.013739904, -0.0013045246, -0.02565323, -0...."
8705,"Как на Rutube смотреть короткие видео, как на ...",ВИДЕО,Воспроизведение видео,ВИДЕО_Воспроизведение видео,"[-0.007107092, 0.004368094, -0.0115768695, -0...."
8706,"Где на Rutube найти короткие видео или раздел,...",ВИДЕО,Воспроизведение видео,ВИДЕО_Воспроизведение видео,"[-0.006600106, -0.00065019954, -0.022671448, -..."


In [2]:
random_state = 42

ALL_COLUMNS = ["Вопрос пользователя", "Классификатор 1 уровня", "Классификатор 2 уровня", "target"]

In [3]:
full = pd.read_csv('/home/rinat/repos/galki_rutube/classification/Сенеж хакатон - full.csv')
full = full.rename(columns={
                              "Вопрос пользователя": "Исходный Вопрос пользователя",
                              "JPT вопрос": "Вопрос пользователя",
                              "класификатор 1": "Классификатор 1 уровня",
                              "класификатор 2": "Классификатор 2 уровня",
                              }
                    )

full['target'] = full['Классификатор 1 уровня'] + '_' + full['Классификатор 2 уровня']

In [ ]:
full.isnull().sum()

In [5]:
data = pd.read_excel('02_Реальные_кейсы.xlsx')[['Вопрос пользователя', 'Классификатор 1 уровня', 'Классификатор 2 уровня']]

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

In [6]:
data['target'] = data['Классификатор 1 уровня'] + '_' + data['Классификатор 2 уровня']

In [8]:
# data['target'].value_counts(dropna=False)

In [ ]:
# data['target'].value_counts()

In [ ]:
print(data.shape[0], full.shape[0])

In [8]:
# s

In [9]:
# data.reset_index(drop=True).to_parquet('train.parquet')

In [ ]:
print(data.shape[0])

In [12]:
data.reset_index(drop=True).to_csv('train.csv')

In [11]:
# train, test = train_test_split(data, test_size=0.35, stratify=data['target'], random_state=42)

In [ ]:
# set(train['target'].unique().tolist()) == set(test['target'].unique().tolist())

In [ ]:
# print(data.shape[0], train.shape[0], test.shape[0])

In [14]:
# train = pd.concat([train, full[ALL_COLUMNS]], axis=0)

In [ ]:
# print(train.shape[0], full.shape[0])

In [16]:
# train['Эмбеддинг'] = list(model.encode(train['Вопрос пользователя'].tolist()))
# test['Эмбеддинг'] = list(model.encode(test['Вопрос пользователя'].tolist()))

In [17]:
# def find_similar_class(query, train_df, model, top_n=1):
#     # Получаем эмбеддинг запроса
#     query_embedding = model.encode([query])
    
#     # Вычисляем косинусное сходство
#     similarities = cosine_similarity(query_embedding, np.array(train_df['Эмбеддинг'].tolist()))
    
#     # Находим индексы самых похожих вопросов
#     similar_indices = np.argsort(similarities[0])[::-1][:top_n]
    
#     # Получаем классы и вопросы похожих вопросов
#     similar_classes = train_df.iloc[similar_indices]['target']
#     similar_questions = train_df.iloc[similar_indices]['Вопрос пользователя']
    
#     # Определяем класс с наибольшим количеством голосов
#     most_common_class = Counter(similar_classes).most_common(1)[0][0]
    
#     return similar_questions, most_common_class

# def predict_classes(test, train, model, top_n):
#     predictions = []
#     similar_examples = []
    
#     for query in test['Вопрос пользователя']:
#         similar_questions, predicted_class = find_similar_class(query, train, model, top_n)
#         predictions.append(predicted_class)
#         # Сохраняем наиболее похожий вопрос
#         similar_examples.append(similar_questions.iloc[0] if not similar_questions.empty else None)
    
#     # Добавляем новые колонки в тестовый датафрейм
#     test['predictions'] = predictions
#     test['similar_examples'] = similar_examples
    
#     return test

In [18]:
# def calculate_metrics(test, column):
#     # accuracy_per_class = test.groupby('Классификатор 1 уровня', as_index=False)[column].apply(
#     #     lambda x: accuracy_score(test.loc[x.index, 'Классификатор 1 уровня'], x)
#     # )

#     overall_accuracy = accuracy_score(test['target'], test[column])
#     print(overall_accuracy)

#     class_report = classification_report(test['target'], test[column], zero_division=0)
#     print(class_report)

In [19]:
# test = predict_classes(test, train, model, 1)

In [ ]:
# calculate_metrics(test, 'predictions')

In [ ]:
# test.loc[test['target'] != test['predictions']]

In [ ]:
# 

In [ ]:
# 

In [19]:
test.loc[test['target'] != test['predictions']].to_csv('wrong_predictions_target.csv')

In [ ]:
# GRID_OF_TOP_N = list(range(1, 21))

# for top_n in GRID_OF_TOP_N:
#     test = predict_classes(test, train, model, top_n)

#     print(F"====================== TOP_N - {top_n} START METRICS REPORT ======================")
#     calculate_metrics(test, 'predictions')
#     print(F"====================== TOP_N - {top_n} END METRICS REPORT ======================")

In [ ]:
# 